In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta, date
import requests

In [2]:
companies = {'NFLX':'netflix', 'YELP':'yelp','AMZN':'amazon','SBUX':'starbucks', 'NKE':'nike', 'TSLA':'tesla'}

In [3]:
companies_list = ['Amazon','nike', 'netflix', 'tesla', 'starbucks', 'yelp']

In [4]:
#collecting the stock data from the alphaadvantage API. only paramatre is the acronym of the company
def get_stock_data(acronym):
    return requests.get(f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={acronym}&outputsize=full&datatype=csv&apikey=SF1XCV4YAFTCHODA')

In [5]:
#function to clean the dataframe received from the API
def clean_stock_dataframe(dataframe):
    change_columns = dataframe.rename(columns = {0:'date', 1:'open', 2:'high', 3:'low',4:'close', 5:'volume'})
    drop_tail = change_columns.drop(change_columns.tail(1).index)
    drop_head = drop_tail.drop(drop_tail.head(1).index)
    return drop_head

In [6]:
#combining the get_stock_data and clean_stock_dataframe to get the companies cleaned dataframe
def company_stock_dataframe(acronym):
    company = get_stock_data(acronym)
    df = pd.DataFrame([x.replace('\r',"").split(',') for x in company.text.split('\n')])
    df['id'] = acronym
    clean_df = clean_stock_dataframe(df)
    return clean_df

In [7]:
#applying the company_stock_dataframe function to a list of companies and saving it
def gathering_all_companies(companies_list):
    comp_df = pd.DataFrame(columns= ['date', 'open', 'high','low', 'close', 'volume', 'name'])
    for company in companies_list:
        new_comp = company_stock_dataframe(company)
        new_comp.dropna(axis = 0, inplace = True)
        comp_df = comp_df.append(new_comp, ignore_index = True)
    comp_df['date'] = comp_df['date'].apply(pd.to_datetime).dt.date
    comp_df.to_csv('../02 CSV_files/stock/stock_data.csv')
    return comp_df

In [8]:
stock_df = gathering_all_companies(companies.keys())

KeyboardInterrupt: 

In [22]:
stock_df = pd.read_csv('../02 CSV_files/stock/stock_data.csv', index_col = 0)
stock_df.drop(columns = 'name', inplace = True)

In [23]:
def preparing_stock_df(df):
    df['name'] = df['id'].apply(lambda x : companies[x] )
    df['stock_result'] = (df['close'] - df['open']) / df['open']
    df['target'] = df['stock_result'].apply(lambda x : 1 if x > 0 else 0)
    df.dropna(axis = 0, inplace = True)
    df.reset_index(inplace = True, drop =True)
    df['date'] = pd.to_datetime(df.date)
    df = df.sort_values(by= 'date')
    return df

In [24]:
prepared_stocks = preparing_stock_df(stock_df)

In [25]:
#creating columns referencing previous rows, in order to see the evolution of the score
def shift_n_rows(df,col_name,n_rows):
    for col in range(1,n_rows+1):
        df[f'{col_name}_day_{col}'] = df[col_name].shift(col)
    return df

In [26]:
#applying the shift function individualy to each company and combining it afterwards into one dataframe
def create_shifted_df(companies, df ,col_name, n_of_shifts):
    shift = {}
    shifted = pd.DataFrame()
    for company in companies:
        new_df = df[df['name'] == company]
        new_df = shift_n_rows(new_df, col_name, n_of_shifts)
        shift[company] = new_df
        shifted = shifted.append(new_df)
    shifted.dropna(axis = 0, inplace = True)
    shifted.date = pd.to_datetime(shifted.date)
    shifted = shifted.reset_index(drop = True)
    shifted.to_csv('../02 CSV_files/csv_finals/final_stocks.csv')
    return shifted

In [27]:
shifted_stocks = create_shifted_df(companies.values(), prepared_stocks, 'stock_result',14)
shifted_stocks = create_shifted_df(companies.values(), shifted_stocks, 'volume',14)
shifted_stocks = create_shifted_df(companies.values(), shifted_stocks, 'high',14)
shifted_stocks = create_shifted_df(companies.values(), shifted_stocks, 'low',14)

C:\Users\usuario\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [28]:
shifted_stocks

,close,date,high,id,low,open,volume,name,stock_result,target,...,low_day_5,low_day_6,low_day_7,low_day_8,low_day_9,low_day_10,low_day_11,low_day_12,low_day_13,low_day_14
0,10.82,2002-08-13,11.27,NFLX,10.560,10.7500,119800,netflix,0.006512,1,...,10.400,10.650,9.89,12.29,12.68,12.51,11.90,11.90,14.49,15.04
1,11.67,2002-08-14,11.67,NFLX,10.800,10.8600,289100,netflix,0.074586,1,...,10.400,10.400,10.65,9.89,12.29,12.68,12.51,11.90,11.90,14.49
2,12.36,2002-08-15,12.75,NFLX,11.660,11.7200,233100,netflix,0.054608,1,...,10.510,10.400,10.40,10.65,9.89,12.29,12.68,12.51,11.90,11.90
3,12.78,2002-08-16,12.94,NFLX,12.000,12.5000,130600,netflix,0.022400,1,...,10.900,10.510,10.40,10.40,10.65,9.89,12.29,12.68,12.51,11.90
4,13.75,2002-08-19,13.78,NFLX,12.850,12.9500,148200,netflix,0.061776,1,...,10.550,10.900,10.51,10.40,10.40,10.65,9.89,12.29,12.68,12.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23850,808.01,2020-05-19,822.07,TSLA,806.080,815.1700,9636522,tesla,-0.008783,0,...,808.000,785.000,787.01,772.35,761.11,762.18,698.00,683.04,763.50,783.16
23851,815.56,2020-05-20,826.00,TSLA,811.800,820.5000,7309271,tesla,-0.006021,0,...,763.300,808.000,785.00,787.01,772.35,761.11,762.18,698.00,683.04,763.50
23852,827.60,2020-05-21,832.50,TSLA,796.000,816.0000,12182524,tesla,0.014216,1,...,764.000,763.300,808.00,785.00,787.01,772.35,761.11,762.18,698.00,683.04
23853,816.88,2020-05-22,831.78,TSLA,812.000,822.1735,9987475,tesla,-0.006438,0,...,786.552,764.000,763.30,808.00,785.00,787.01,772.35,761.11,762.18,698.00
